In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### x_w3_L9(last lec)-mlt-dip-iitm

# Multi-output/Multi-label Regression
In case of multi-output regression, there are more than one output labels, all of which are real numbers.

## Training data

let's generate synthetic data for demonstrating the training set in multi-output regression using make_regression dataset generation function from sklearn library.

In [2]:
from sklearn.datasets import make_regression
X, y, coef = make_regression(n_samples=100, n_features=10, n_informative=10, bias=1, n_targets=5, shuffle=True, coef=True, random_state=42)

In [3]:
print(X.shape)
print(y.shape)

(100, 10)
(100, 5)


Let's examine first five examples in terms of their features and labels:

In [4]:
print("Sample training examples:\n ", X[:5])
print("\n")
print("Corresponding labels:\n ", y[:5])

Sample training examples:
  [[-2.07339023 -0.37144087  1.27155509  1.75227044  0.93567839 -1.40751169
  -0.77781669 -0.34268759 -1.11057585  1.24608519]
 [-0.90938745 -1.40185106 -0.50347565 -0.56629773  0.09965137  0.58685709
   2.19045563  1.40279431 -0.99053633  0.79103195]
 [-0.18565898 -1.19620662 -0.64511975  1.0035329   0.36163603  0.81252582
   1.35624003 -1.10633497 -0.07201012 -0.47917424]
 [ 0.03526355  0.21397991 -0.57581824  0.75750771 -0.53050115 -0.11232805
  -0.2209696  -0.69972551  0.6141667   1.96472513]
 [-0.51604473 -0.46227529 -0.8946073  -0.47874862  1.25575613 -0.43449623
  -0.30917212  0.09612078  0.22213377  0.93828381]]


Corresponding labels:
  [[-133.15919852  -88.95797818   98.19127175   25.68295511 -132.79294654]
 [-110.38909784  146.04459736 -169.58916067  118.96066861 -177.08414159]
 [ -97.80350267    4.32654061  -87.56082281   -5.58466452    6.36897388]
 [  25.39024616  -70.41180117  186.15213706  132.77153362   53.42301307]
 [-140.61925153  -53.8700783

and the coefficients or weight vector used for generating this dataset is  

In [5]:
coef

array([[93.62122462,  5.19712837, 54.12963353, 70.90605195, 87.09691237],
       [89.48166561, 54.75923762, 81.729777  , 45.23182845, 64.35776952],
       [46.26229567, 86.82725054, 72.71690698, 74.27065212, 42.54933344],
       [71.92017783, 22.84547413, 99.63339161, 97.47931621, 65.03256863],
       [19.95424509, 68.02282424,  7.2198409 ,  3.06525022, 25.76828885],
       [52.64026609, 73.15895218,  8.1629982 ,  6.0352084 , 24.7103234 ],
       [15.95446801, 87.17835666, 21.92139874, 97.58652558, 33.68957918],
       [71.40869321, 80.17280831, 33.94501925, 81.48251137,  8.01148464],
       [18.21179157, 78.96985071, 65.87077755, 49.81957165, 55.53635509],
       [16.74825823, 10.45678403, 63.64302495, 70.64757265,  3.15861448]])

## [Preprocessing: Dummy feature and train-test split]

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
def add_dummy_feature(X):
    # np.column_stack((np.ones(x.shape[0]) x))
    X_dummyFeature = np.column_stack((np.ones(X.shape[0]), X))
    return X_dummyFeature

def trainTestSplit(X, y):
    return train_test_split(X,y, test_size=.2,random_state=42 )

def preprocess(X, y):
    X_withdummyfeature = add_dummy_feature(X)
    X_train, X_test, y_train, y_test = trainTestSplit(X_withdummyfeature, y)
    return (X_train, X_test, y_train, y_test)

In [8]:
X_train, X_test, y_train, y_test = preprocess(X, y)

## Model

There are two options for modeling this problem:
1. Solve k independent linear regression problems. Gives some flexibility in using different representation for each problem.
2. Solve a joint learning problem as outlined in the equation above. We would pursue this approach.

## Loss

Loss function(loss): J(w) = (1/2)$(Xw-y)^T  (Xw - y)$

## Optimization
1. Normal equation
2. Gradient descent and its variations


## Evaluation
RMSE or Loss

# Linear regression implementation

In [9]:
class LinReg(object):
    '''
    Linear regression model
    -----------------------
    y = X@w
    X: A feature matrix
    w: weight vector
    y: label vector
    
    '''
    def __init__(self):
        self.t0 = 200
        self.t1 = 100000
        
    def predict(self, X:np.ndarray) -> np.ndarray:
        '''Prediction of output label for a given input.
        Args:
            X: Feature matrix for given inputs.
        Returns:
            y: Output label vector as predicted by the given model.
        
        '''
        y = X @ self.w
        return y
    
    def loss(self, X:np.ndarray, y:np.ndarray) -> float:
        '''Calculate loss for a model based on known labels
        Args:
            X: Feature matrix for given inputs.
            y: Output label vector as predicted by the given model.
        Returns:
            Loss
        '''
        e = y - self.predict(X)
        return (1/2) * (np.transpose(e) @ e)
    
    def rmse(self, X:np.ndarray, y:np.ndarray) -> float:
        '''Calculates root mean squared error of prediction w.r.t. actual label.
        Args:
            X: Feature matrix for given inputs.
            y: Output label vector as predicted by the given model.
        Returns:
            Loss
        '''
        return np.sqrt((2/X.shape[0]) * self.loss(X, y))
    
    def fit(self, X:np.ndarray, y:np.ndarray) -> np.ndarray:
        '''Estimates parameters of the linear regression model with normal equation.
        Args:
            X: Feature matrix for given inputs.
            y: Output label vector as predicted by the given model.
        Returns:
            Weight vector
        '''
        self.w = np.linalg.pinv(X) @ y
        return self.w
    
    def calculate_gradient(self, X:np.ndarray, y:np.ndarray)->np.ndarray:
        '''Calculates gradients of loss function w.r.t. weight vector on training set.
        Args:
            X: Feature matrix for given inputs.
            y: Output label vector as predicted by the given model.
        Returns:
            A vector of gradients
        
        '''
        return np.transpose(X)@(self.predict(X) - y)
    
    def update_weights(self, grad:np.ndarray, lr:float) -> np.ndarray:
        '''Updates the weights based on the gradient of loss function.
        Weight updates are carried out with the following formula:
            w_new := w_old - lr*grad
        Args:
            2. grad: gradient of loss w.r.t. w
            3. lr: learning rate
        Returns:
            Updated weight vector
        
        '''
        return (self.w - lr*grad)
    
    def learning_schedule(self, t):
        return self.t0 / (t + self.t1)
    
    def gd(self, X:np.ndarray, y:np.ndarray, num_epochs:int, lr:float) -> np.ndarray:
        '''Estimates parameters of linear regression model through gradient descent.
        Args:
            X: Feature matrix for given inputs.
            y: Output label vector as predicted by the given model.
            num_epochs: Number of training steps
            lr: learning rate
       Returns:
            Weight vector: Final weight vector
        
        '''
        self.w = np.zeros((X.shape[1], y.shape[1]))
        self.w_all = []
        self.err_all = []
        for i in np.arange(0, num_epochs):
            dJdW = self.calculate_gradient(X, y)
            self.w_all.append(self.w)
            self.err_all.append(self.loss(X, y))
            self.w = self.update_weights(dJdW, lr)
        return self.w
    
    def mbgd(self, X:np.ndarray, y:np.ndarray, 
                   num_epochs:int, batch_size:int) -> np.ndarray:
        '''Estimates parameters of linear regression model through gradient descent.

        Args:
            X: Feature matrix of training data.
            y: Label vector for training data
            num_epochs: Number of training steps
            batch_size: Number of examples in a batch
        Returns:
            Weight vector: Final weight vector
           

        '''
        self.w = np.zeros((X.shape[1]))
        self.w_all = [] # all params across iterations.
        self.err_all = [] # error across iterations
        mini_batch_id = 0

        for epoch in range(num_epochs):
            shuffled_indices = np.random.permutation(X.shape[0])
            X_shuffled = X[shuffled_indices]
            y_shuffled = y[shuffled_indices]
            for i in range(0, X.shape[0], batch_size):
                mini_batch_id += 1
                xi = X_shuffled[i:i+minibatch_size]
                yi = y_shuffled[i:i+minibatch_size]
                
                self.w_all.append(self.w)
                self.err_all.append(self.loss(xi, yi))

                dJdW = 2/batch_size * self.calculate_gradient(xi, yi)
                self.w = self.update_weights(dJdW, self.learning_schedule(mini_batch_id))
               

        return self.w
    
    def sgd(self, X:np.ndarray, y:np.ndarray, 
                   num_epochs:int, batch_size:int) -> np.ndarray:
        '''Estimates parameters of linear regression model through gradient descent.

        Args:
            X: Feature matrix of training data.
            y: Label vector for training data
            num_epochs: Number of training steps
            batch_size: Number of examples in a batch
        Returns:
            Weight vector: Final weight vector
            

        '''
        self.w = np.zeros((X.shape[1]))
        self.w_all = [] # all params across iterations.
        self.err_all = [] # error across iterations
        mini_batch_id = 0

        for epoch in range(num_epochs):
            for i in range(X.shape[0]):
                random_index = np.random.randint(X.shape[0])
                xi = X_shuffled[random_index:random_index+1]
                yi = y_shuffled[random_index:random_index+1]
                
                self.w_all.append(self.w)
                self.err_all.append(self.loss(xi, yi))

                gradients = 2 * self.calculate_gradient(xi, yi)
                lr = self.learning_schedule(epoch * X.shape[0] + i)
                self.w = self.update_weights(gradients, lr)
               

        return self.w
    

In [10]:
lin_reg = LinReg()
w = lin_reg.fit(X_train, y_train)

# Check if the weight vector si same as the coefficient vector used fo rmaking the data:
np.testing.assert_almost_equal(w[1:, :], coef, decimal=2)

Let's check the estimated weight vector

In [11]:
w

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [93.62122462,  5.19712837, 54.12963353, 70.90605195, 87.09691237],
       [89.48166561, 54.75923762, 81.729777  , 45.23182845, 64.35776952],
       [46.26229567, 86.82725054, 72.71690698, 74.27065212, 42.54933344],
       [71.92017783, 22.84547413, 99.63339161, 97.47931621, 65.03256863],
       [19.95424509, 68.02282424,  7.2198409 ,  3.06525022, 25.76828885],
       [52.64026609, 73.15895218,  8.1629982 ,  6.0352084 , 24.7103234 ],
       [15.95446801, 87.17835666, 21.92139874, 97.58652558, 33.68957918],
       [71.40869321, 80.17280831, 33.94501925, 81.48251137,  8.01148464],
       [18.21179157, 78.96985071, 65.87077755, 49.81957165, 55.53635509],
       [16.74825823, 10.45678403, 63.64302495, 70.64757265,  3.15861448]])

The weight vectors are along the column.

In [12]:
w = lin_reg.gd(X_train, y_train, num_epochs=100, lr=0.01)
np.testing.assert_almost_equal(w[1:, :], coef, decimal=2)